In [1]:
from mxnet import nd
from mxnet.gluon import nn

In [2]:
#dense layer with two output units
layer = nn.Dense(2)
layer

Dense(None -> 2, linear)

In [3]:
#Initialize its weights with the default initialization method, which draws random values uniformly from [-0.7,0.7]
layer.initialize()

In [5]:
#Forward pass with random data. We create a (3,4)shape random x and feed into the layer to compute the output
x = nd.random.uniform(-1,1,(3,4))
layer(x)


[[ 0.00986117 -0.04696713]
 [-0.00033285 -0.03279176]
 [-0.00225417 -0.05342289]]
<NDArray 3x2 @cpu(0)>

In [6]:
#Access weight
layer.weight.data()


[[ 0.00952624 -0.01501013  0.05958354  0.04705103]
 [-0.06005495 -0.02276454 -0.0578019   0.02074406]]
<NDArray 2x4 @cpu(0)>

In [8]:
#Chain layers into a neural network
#During forward pass, we run layers sequentially one by one. The following code implements a famous network called LeNet through nn .Sequential
net  = nn.Sequential()
#Add a sequence of layers
net.add(#similar to dense, it is not necessarily to identify the input channels
nn.Conv2D(channels=6, kernel_size=5, activation = 'relu'),
#One can also use a tuple to specify non-symmetric pool and stride sizes
nn.MaxPool2D(pool_size=2, strides=2),
nn.Conv2D(channels=16, kernel_size=3, activation='relu'),
nn.MaxPool2D(pool_size=2, strides=2),
    #The dense layer will automatically reshape the 4-D output of the last
    #Max pooling layer into the 2-D shape: (x.shape[0], x.size/x.shape[0])
nn.Dense(120, activation = "relu"),
nn.Dense(84, activation = "relu"),
nn.Dense(10))
net

Sequential(
  (0): Conv2D(None -> 6, kernel_size=(5, 5), stride=(1, 1), Activation(relu))
  (1): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False, global_pool=False, pool_type=max, layout=NCHW)
  (2): Conv2D(None -> 16, kernel_size=(3, 3), stride=(1, 1), Activation(relu))
  (3): MaxPool2D(size=(2, 2), stride=(2, 2), padding=(0, 0), ceil_mode=False, global_pool=False, pool_type=max, layout=NCHW)
  (4): Dense(None -> 120, Activation(relu))
  (5): Dense(None -> 84, Activation(relu))
  (6): Dense(None -> 10, linear)
)

In [9]:
net.initialize()
#Input shape is(batch_size, color_channels, height, width)
x = nd.random.uniform(shape = (4,1,28,28))
y = net(x)
y.shape

(4, 10)

In [10]:
(net[0].weight.data().shape, net[5].bias.data().shape)

((6, 1, 5, 5), (84,))

In [24]:
#Create A Neural Network Flexibly
class MixMLP(nn.Block):
    def __init__(self, **kwargs):
        # Run `nn.Block`'s init method
        super(MixMLP, self).__init__(**kwargs)
        self.blk = nn.Sequential()
        self.blk.add(nn.Dense(3, activation='relu'),
                     nn.Dense(4, activation='relu'))
        self.dense = nn.Dense(5)
    def forward(self, x):
        y = nd.relu(self.blk(x))
        print(y)
        return self.dense(y)

net = MixMLP()
net

MixMLP(
  (blk): Sequential(
    (0): Dense(None -> 3, Activation(relu))
    (1): Dense(None -> 4, Activation(relu))
  )
  (dense): Dense(None -> 5, linear)
)

In [25]:
net.initialize()
x = nd.random.uniform(shape=(2,2))
net(x)


[[0.0000000e+00 0.0000000e+00 2.1124955e-05 6.4140237e-05]
 [0.0000000e+00 0.0000000e+00 2.2888425e-04 5.9812429e-04]]
<NDArray 2x4 @cpu(0)>



[[ 1.1762700e-06  4.1317876e-06 -3.8491830e-06  3.0992901e-06
  -1.3130854e-06]
 [ 1.3158093e-05  3.9258037e-05 -3.8034632e-05  2.8098166e-05
  -1.1850577e-05]]
<NDArray 2x5 @cpu(0)>

In [27]:
#Access a particular weight class
net.blk[1].weight.data()


[[-0.04485548  0.00594983 -0.06654498]
 [ 0.04964591 -0.06058505  0.03413684]
 [ 0.02511498 -0.00299651 -0.00648244]
 [ 0.02479142  0.00512109  0.01498631]]
<NDArray 4x3 @cpu(0)>